In [ ]:
# prompt: グーグルドライブと連結

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,MinMaxScaler
from imblearn.over_sampling import SMOTE
from category_encoders.target_encoder import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
# import optuna
# from optuna import Trial
from imblearn.combine import SMOTEENN
from collections import Counter

from xgboost import XGBClassifier

from lightgbm import LGBMClassifier
from imblearn.over_sampling import BorderlineSMOTE
from scipy.stats import uniform, randint
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier , GradientBoostingClassifier , ExtraTreesClassifier , VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV , cross_val_score , StratifiedKFold , learning_curve
from sklearn.impute import KNNImputer

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
client = pd.read_csv("/content/drive/MyDrive/A_comp/Client.csv")
record = pd.read_csv("/content/drive/MyDrive/A_comp/Record.csv")

In [ ]:
pd.set_option('display.max_info_columns', 1000)
pd.set_option('display.max_columns', 1000)

# EDA

In [ ]:
print(client.info())
print('-----------------------------')
record.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 50 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   uniqsubs          100000 non-null  int64  
 1   actvsubs          100000 non-null  int64  
 2   new_cell          100000 non-null  object 
 3   crclscod          100000 non-null  object 
 4   asl_flag          100000 non-null  object 
 5   totcalls          100000 non-null  int64  
 6   totmou            100000 non-null  float64
 7   totrev            100000 non-null  float64
 8   adjrev            100000 non-null  float64
 9   adjmou            100000 non-null  float64
 10  adjqty            100000 non-null  int64  
 11  avgrev            100000 non-null  float64
 12  avgmou            100000 non-null  float64
 13  avgqty            100000 non-null  float64
 14  avg3mou           100000 non-null  int64  
 15  avg3qty           100000 non-null  int64  
 16  avg3rev           100

In [ ]:
record.shape

(100000, 51)

In [ ]:
all_df = pd.merge(client, record, on='Customer_ID', how='outer')
all_df_ = all_df.copy()

## outlier처리

In [ ]:
cleandf = all_df.copy()
cleandf = all_df[all_df["totrev"] < 20000]
cleandf = cleandf[cleandf["totmou"] < 200000]
cleandf = cleandf[cleandf["uniqsubs"] < 150]
cleandf = cleandf[all_df["actvsubs"] < 30]
cleandf = cleandf[cleandf["adjmou"] < 200000]

all_df_ = cleandf

#hnd_webcap
all_df_["hnd_webcap"].replace("UNKW",np.nan,inplace= True)

## encoding

In [ ]:
#2
E = LabelEncoder()
Larray = ["asl_flag","refurb_new","kid0_2","kid3_5","kid6_10",
          "kid11_15","kid16_17","creditcd"]
for i in range(len(Larray)):
    Labeled = E.fit_transform(all_df_[Larray[i]])
    all_df_[Larray[i]] = Labeled


In [ ]:
# some

ColumnsOneHot = ["infobase","dwlltype","ownrent","hnd_webcap","new_cell","prizm_social_one",
                 "dualband","marital","HHstatin"]
encoder = OneHotEncoder(sparse_output = False)

client_encoded = pd.get_dummies(all_df_[ColumnsOneHot])

all_df_.drop(columns=ColumnsOneHot, inplace=True)
all_df_ = pd.concat([all_df_, client_encoded], axis=1)

In [ ]:
# Target Encoding
encoder = TargetEncoder()
Targetcolumns = ["ethnic","dwllsize","crclscod","area"]
for j in Targetcolumns:
  all_df_[j] = encoder.fit_transform(all_df_[j] ,all_df_["churn"])

In [ ]:
null_counts = all_df_.isnull().sum()
null_counts[null_counts > 0]

,0
avg6mou,2839
avg6qty,2839
avg6rev,2839
hnd_price,847
phones,1
models,1
truck,1732
rv,1732
lor,30189
adults,23018


## 결손값 보관

In [ ]:
all_df_except_null = all_df_[all_df_['phones'].notnull()&all_df_["rev_Mean"].notnull()&all_df_['change_mou'].notnull()&all_df_['hnd_price'].notnull()]

In [ ]:
## 삭제 처리
all_df_except_null["forgntvl"].fillna(0.0, inplace=True)
all_df_except_null["rv"].fillna(0.0, inplace=True)
all_df_except_null["truck"].fillna(0.0, inplace=True)


正規分布補完

In [ ]:
all_df_except_null_avg6mou_mean = all_df_except_null["avg6mou"].mean()
all_df_except_null_avg6qty_mean = all_df_except_null["avg6qty"].mean()
all_df_except_null_avg6rev_mean = all_df_except_null["avg6rev"].mean()

all_df_except_null_avg6mou_std = all_df_except_null["avg6mou"].std()
all_df_except_null_avg6qty_std = all_df_except_null["avg6qty"].std()
all_df_except_null_avg6rev_std = all_df_except_null["avg6rev"].std()

all_df_except_null_2 = all_df_except_null.copy()

# 欠損値の位置を取得
missing_indices = all_df_except_null_2[all_df_except_null_2['avg6mou'].isna()].index

print(missing_indices.shape)

# 正規分布からランダムに値をサンプリングして補完
for index in missing_indices:
    all_df_except_null_2.at[index, 'avg6mou'] = np.random.normal(all_df_except_null_avg6mou_mean, all_df_except_null_avg6mou_std)
    all_df_except_null_2.at[index, 'avg6qty'] = np.random.normal(all_df_except_null_avg6qty_mean, all_df_except_null_avg6qty_std)
    all_df_except_null_2.at[index, 'avg6rev'] = np.random.normal(all_df_except_null_avg6rev_mean, all_df_except_null_avg6rev_std)

(2730,)


In [ ]:
all_df_except_null_counts2 = all_df_except_null_2.isnull().sum()
all_df_except_null_counts2[all_df_except_null_counts2 > 0]

,0
lor,29634
adults,22579
income,24948
numbcars,48424


KNNimputer보관

In [ ]:
imputer = KNNImputer(n_neighbors=3)
all_df_except_null_3_array = imputer.fit_transform(all_df_except_null_2)

all_df_except_null_3 = pd.DataFrame(all_df_except_null_3_array, columns=all_df_except_null_2.columns)

all_df_except_null_3.to_csv('./submission.csv', index=False)